#QSVC - DGA Botnet detection

Madjid Tehrani
Cybersec-DMS (DMS GmbH)
https://www.cybersec-dms.com/
Copyright © 2023, DMS GmbH. All rights reserved. Permission is hereby granted, free of charge, to any person obtaining a copy of this
software and associated documentation files (the "Software"), for academic research purposes, subject to the following conditions:
the above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.   


Let's generate some data:

In [ ]:
!pip install qiskit-machine-learning

In [ ]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from azure.quantum.qiskit import AzureQuantumProvider
from qiskit import QuantumCircuit, execute, Aer
from qiskit import *
from qiskit.utils import QuantumInstance
from azure.quantum import Workspace

provider = AzureQuantumProvider (
    resource_id = "/subscriptions/yourID/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/yourWorkspace",
    location = "eastus"
)

print("This workspace's targets:")
for backend in provider.backends():
    print("- " + backend.name())

workspace = Workspace (
    subscription_id = "yourID",
    resource_group = "AzureQuantum",
    name = "yourWorkspace",
    location = "eastus"

)

In [ ]:
# Create IonQ simulator and QPU backends
ionq_simulator_backend = provider.get_backend("ionq.simulator")
ionq_qpu_backend = provider.get_backend("ionq.qpu")

# Create Rigetti simulator and QPU backends
rigetti_simulator_backend = provider.get_backend("rigetti.sim.qvm")
rigetti_qpu_backend = provider.get_backend("rigetti.qpu.aspen-m-3")

# Create Quantinuum simulator and QPU backends
Quantinuum_simulator_backend = provider.get_backend("quantinuum.sim.h1-2sc")
Quantinuum_qpu_backend = provider.get_backend("quantinuum.qpu.h1-2")

# Create QASM simulator
AER_backend = Aer.get_backend('qasm_simulator')

In [ ]:
import os
!wget https://aq5efd7d2644dd406cb3ec2d.blob.core.windows.net/dga/BotnetDgaDataset_1000.csv
files = os.listdir(os.curdir)
for file in files:
    print(file)

--2023-08-16 17:20:05--  https://aq5efd7d2644dd406cb3ec2d.blob.core.windows.net/dga/BotnetDgaDataset_1000.csv
Resolving aq5efd7d2644dd406cb3ec2d.blob.core.windows.net (aq5efd7d2644dd406cb3ec2d.blob.core.windows.net)... 52.239.169.4
Connecting to aq5efd7d2644dd406cb3ec2d.blob.core.windows.net (aq5efd7d2644dd406cb3ec2d.blob.core.windows.net)|52.239.169.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75901 (74K) [text/csv]
Saving to: ‘BotnetDgaDataset_1000.csv’

BotnetDgaDataset_10 100%[===================>]  74.12K   439KB/s    in 0.2s    

2023-08-16 17:20:06 (439 KB/s) - ‘BotnetDgaDataset_1000.csv’ saved [75901/75901]

.config
BotnetDgaDataset_1000.csv
sample_data


In [ ]:
import csv
import os
import numpy as np
from sklearn.datasets import make_blobs
datafilename="BotnetDgaDataset_1000.csv"
resultname="result_BotnetDgaDataset_pegasos_1000.txt"
cwd=os.getcwd()
mycsv=cwd+"/"+datafilename
print(mycsv)
def load_data(filepath):
    with open(filepath) as csv_file:
        data_file = csv.reader(csv_file)
        temp = next(data_file)
        n_samples = 1000
        n_features = 7
        data = np.empty((n_samples, n_features))
        target = np.empty((n_samples,), dtype=int)

        for i, ir in enumerate(data_file):
            data[i] = np.asarray(ir[:-1], dtype=np.float64)
            target[i] = np.asarray(ir[-1], dtype=int)

    return data, target
features, labels = load_data(mycsv)

/content/BotnetDgaDataset_1000.csv


In [ ]:
import qiskit.tools.jupyter

%qiskit_version_table

In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

features = MinMaxScaler(feature_range=(0, np.pi)).fit_transform(features)

train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, train_size=700, shuffle=False
)


In [ ]:
from qiskit import BasicAer
from qiskit.circuit.library import ZFeatureMap
from qiskit.utils import algorithm_globals

from qiskit_machine_learning.kernels import FidelityQuantumKernel
num_qubits = 7
algorithm_globals.random_seed = 12345

feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=1)

qkernel = FidelityQuantumKernel(feature_map=feature_map)

In [ ]:
from qiskit_machine_learning.algorithms import QSVC
import time
from qiskit.algorithms.optimizers import SLSQP, SPSA
from qiskit.circuit.library import ZFeatureMap

num_features = features.shape[1]

feature_map = ZFeatureMap(feature_dimension=num_features, reps=1)
qsvc = QSVC(quantum_kernel=qkernel)


import time
QSVC_start=time.perf_counter()
# training
qsvc.fit(train_features, train_labels)

# testing
qsvc_score = qsvc.score(test_features, test_labels)
QSVC_end=time.perf_counter()

print(f"QSVC Accuracy: {qsvc_score}")
print(QSVC_end-QSVC_start)

QSVC Accuracy: 0.8633333333333333
3772.9200665050002
